In [2]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import joblib

In [4]:
import pandas as pd
from utils.preprocess import treat_dataset_pandas_init, build_preprocessing_pipeline,INIT_NUMERICAL_COLS, numerical_features

from utils.metrics import rmsle_metric

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import root_mean_squared_log_error
from sklearn.ensemble import StackingRegressor, HistGradientBoostingRegressor
import numpy as np
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor

from catboost import CatBoostRegressor
from utils.categoricals import CategoricalToStringTransformer, CategoricalEncoder, CatogoricalUnknownTransformer



In [5]:
train_data = pd.read_csv("playground-series-s4e12/train.csv")#.sample(frac=0.1)
test_data = pd.read_csv("playground-series-s4e12/test.csv")

treated_dataset = treat_dataset_pandas_init(train_data, process_as_category=True)
#non_loged_train, non_loged_test = joblib.load("cat_non_loged_local.pkl")
non_loged_train, non_loged_test = joblib.load("cat_non_loged.pkl")

treated_dataset['non_log_oof_prediction'] = non_loged_train

X_train = treated_dataset.drop(columns=["Premium Amount"])
y_train = treated_dataset["Premium Amount"]


In [6]:
root_mean_squared_log_error(treated_dataset['Premium Amount'], treated_dataset['non_log_oof_prediction'])

1.1408728857820265

In [8]:
treated_dataset

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,...,Income_per_Dependent,CreditScore_InsuranceDuration,Health_Risk_Score,Credit_Health_Score,Health_Age_Interaction,Claims v Duration,Health vs Claims,Cat Credit Score,Int Credit Score,non_log_oof_prediction
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,...,5024.5,1860.0,3.870062,8406.738970,429.376453,0.400000,7.532920,372.0,372.0,1198.816057
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,...,7919.5,1388.0,4.221513,10805.393307,607.219509,0.500000,7.784865,694.0,694.0,953.272016
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,...,6400.5,NaN,2.641123,NaN,1085.083634,0.333333,23.588775,<NA>,NaN,1104.083469
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,...,47285.0,367.0,4.453093,4014.298906,229.701027,1.000000,5.469072,367.0,367.0,1276.605480
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,...,19825.5,2392.0,3.981195,12184.903989,427.897966,0.000000,20.376094,598.0,598.0,1273.640800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,...,27316.0,1116.0,4.311355,5123.521323,495.824644,NaN,NaN,372.0,372.0,1234.252578
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,...,NaN,2388.0,4.425826,6855.638903,620.108041,NaN,NaN,597.0,597.0,1213.428470
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,...,51884.0,NaN,4.263777,NaN,279.764902,0.000000,14.724469,<NA>,NaN,544.038304
1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,Premium,...,NaN,1628.0,4.072631,7548.784101,1020.105960,0.250000,9.273691,407.0,407.0,1235.857653


In [9]:
from sklearn.model_selection import KFold


In [10]:

# Define a 60-40 split
#splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
splitter = KFold(n_splits=5, shuffle=True, random_state=42)


In [11]:
from sklearn.metrics import mean_squared_log_error


In [12]:
def rmsle(y_true, y_pred):
    y_pred = np.maximum(0, y_pred)  # Clip predicted values to be non-negative

    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [15]:
xgboost_params = {'learning_rate': 0.04656560304624032, 'max_depth': 10, 'min_child_weight': 50, 'subsample': 0.9730484625285342, 'colsample_bytree': 0.5743561244230219, 'gamma': 7.9578377008338235, 'lambda': 9.268384283962487, 'alpha': 9.9663808717552, 'n_estimators': 754}

In [16]:
for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    y_train_fold = np.log1p(y_train_fold)
    y_val_fold = np.log1p(y_val_fold)
    
    full_pipeline = Pipeline([
        #("categorical_encoder", CategoricalEncoder()),
        ("model",xgb.XGBRegressor(enable_categorical=True, **xgboost_params, device="cuda", tree_method="hist"))
    ])
    full_pipeline.fit(X_train_fold, y_train_fold)
    
    train_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_train_fold)), np.expm1(y_train_fold))
    val_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_val_fold)), np.expm1(y_val_fold))

    print(f"Train: {train_score}, Test: {val_score}")

Starting


d:\Development\insurance\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [00:26:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Train: 1.01508361253977, Test: 1.0336200796724437
Starting
Train: 1.0153803688262613, Test: 1.032450494995619
Starting
Train: 1.0148886273675428, Test: 1.034428999931049
Starting
Train: 1.015938982968538, Test: 1.031844295874394
Starting
Train: 1.0151394190373497, Test: 1.0336337284126291


In [17]:
lgbm_params = {'boosting_type': 'gbdt', 'objective': 'regression', 'learning_rate': 0.0239614211663401, 'num_boost_round': 826, 'num_leaves': 169, 'max_depth': 34, 'feature_fraction': 0.5772473473750267, 'bagging_fraction': 0.5660377739630706, 'bagging_freq': 2, 'min_child_samples': 32, 'min_split_gain': 0.04367672923191071, 'min_child_weight': 3.793554610841873, 'lambda_l1': 4.429544342866135, 'lambda_l2': 2.0212763479020808, 'cat_smooth': 26.99333583992606, 'max_bin': 168, 'verbose': -1, 'random_state': 41}


In [18]:
for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    y_train_fold = np.log1p(y_train_fold)
    y_val_fold = np.log1p(y_val_fold)
    
    full_pipeline = Pipeline([
        ("model",LGBMRegressor(**lgbm_params))
    ])
    full_pipeline.fit(X_train_fold, y_train_fold)
    
    
    train_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_train_fold)), np.expm1(y_train_fold))
    val_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_val_fold)), np.expm1(y_val_fold))

    print(f"Train: {train_score}, Test: {val_score}")

Starting


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train: 0.9910957591564978, Test: 1.0347058373067353
Starting


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train: 0.9913558298300467, Test: 1.0336348923577352
Starting


...

Train: 0.9906868469726866, Test: 1.0352591155530728
Starting


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train: 0.9913053658783485, Test: 1.033069399644675
Starting


d:\Development\insurance\venv\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Train: 0.9909832782721372, Test: 1.0347993281192245


In [23]:
catboost_params = {'learning_rate': 0.30849158244687513, 'iterations': 74, 'depth': 7, 'l2_leaf_reg': 0.9408913103331975, 'min_data_in_leaf': 43, 'subsample': 0.6707321357155331, 'colsample_bylevel': 0.8996427501620363}

In [24]:
for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx].copy(), X_train.iloc[val_idx].copy()
    y_train_fold, y_val_fold = y_train.iloc[train_idx].copy(), y_train.iloc[val_idx].copy()
    
    y_train_fold = np.log1p(y_train_fold)
    y_val_fold = np.log1p(y_val_fold)
    
    categorical_features = [col for col in X_train_fold.columns if X_train_fold[col].dtype == "category"]
    
    full_pipeline = Pipeline([
        ("categorical_to_string", CatogoricalUnknownTransformer()),
        ("model",CatBoostRegressor(**catboost_params, cat_features=categorical_features))
    ])
    full_pipeline.fit(X_train_fold, y_train_fold)
    
    train_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_train_fold)), np.expm1(y_train_fold))
    val_score = root_mean_squared_log_error(np.expm1(full_pipeline.predict(X_val_fold)), np.expm1(y_val_fold))

    print(f"Train: {train_score}, Test: {val_score}")

Starting
0:	learn: 1.0696023	total: 223ms	remaining: 16.3s
1:	learn: 1.0559500	total: 389ms	remaining: 14s
2:	learn: 1.0483943	total: 554ms	remaining: 13.1s
3:	learn: 1.0444098	total: 705ms	remaining: 12.3s
4:	learn: 1.0413685	total: 887ms	remaining: 12.2s
5:	learn: 1.0396677	total: 1.04s	remaining: 11.8s
6:	learn: 1.0383621	total: 1.19s	remaining: 11.4s
7:	learn: 1.0376821	total: 1.33s	remaining: 11s
8:	learn: 1.0372948	total: 1.5s	remaining: 10.9s
9:	learn: 1.0366547	total: 1.66s	remaining: 10.6s
10:	learn: 1.0362325	total: 1.81s	remaining: 10.3s
11:	learn: 1.0359433	total: 1.95s	remaining: 10.1s
12:	learn: 1.0357044	total: 2.08s	remaining: 9.76s
13:	learn: 1.0355004	total: 2.21s	remaining: 9.48s
14:	learn: 1.0353417	total: 2.36s	remaining: 9.27s
15:	learn: 1.0352423	total: 2.48s	remaining: 8.99s
16:	learn: 1.0349886	total: 2.62s	remaining: 8.78s
17:	learn: 1.0349217	total: 2.75s	remaining: 8.55s
18:	learn: 1.0346380	total: 2.88s	remaining: 8.34s
19:	learn: 1.0344495	total: 3.01s	rem

# Stacking 

In [22]:
results = []
for train_idx, val_idx in splitter.split(X_train):
    print("Starting")
    # Prepare data for LightGBM
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    y_train_fold = np.log1p(y_train_fold)
    y_val_fold = np.log1p(y_val_fold)

    categorical_features = [col for col in X_train_fold.columns if X_train_fold[col].dtype == "category"]
        
    catboost_model = Pipeline([
        ("categorical_to_string", CatogoricalUnknownTransformer()),
        ("model",CatBoostRegressor(**catboost_params, cat_features=categorical_features))
    ])
    print("Fitting catboost - Start", datetime.now())
    catboost_model.fit(X_train_fold, y_train_fold)
    print("Fitting catboost - End", datetime.now())
    
    linear_model_pipeline = Pipeline([
        ("preprocessing", build_preprocessing_pipeline(train_data=X_train_fold, imputation_method="simple", binning_method="opti-means", categorical_to_string=True)),
        ("model", LinearRegression())
    ])
    
    hist_pipeline = Pipeline([
        ("preprocessing", build_preprocessing_pipeline(train_data=X_train_fold, imputation_method="simple", categorical_to_string=True)),
        ("model", HistGradientBoostingRegressor())
    ])
    
    xgboost = xgb.XGBRegressor(enable_categorical=True, eta=0.1, colsample_bytree=0.9, n_estimators=100, device="cuda", tree_method="hist")
    print("Fitting xgboost - Start", datetime.now())
    xgboost.fit(X_train_fold, y_train_fold)
    print("Fitting xgboost - End", datetime.now())
    
    xgboost_rf = xgb.XGBRFRegressor(enable_categorical=True, eta=0.1, colsample_bytree=0.9, n_estimators=100, device="cuda", tree_method="hist")
    print("Fitting xgboost_rf - Start", datetime.now())
    xgboost_rf.fit(X_train_fold, y_train_fold)
    print("Fitting xgboost_rf - End", datetime.now())
    
    lgbm = LGBMRegressor(learning_rate=0.1, colsample_bytree=0.9, n_estimators=100)
    print("Fitting lgbm - Start", datetime.now())
    lgbm.fit(X_train_fold, y_train_fold)
    print("Fitting lgbm - End", datetime.now())

    stack_model = StackingRegressor(
        estimators=[
            #("linear_model", linear_model_pipeline),
            ("xgboost", xgboost),
            ("xgboost_rf", xgboost_rf),
            ("lgbm", lgbm),
            ("catboost", catboost_model)
        ],
        #final_estimator=BayesianRidge(),
        cv='prefit',
        final_estimator=xgb.XGBRegressor(enable_categorical=True, eta=0.01, colsample_bytree=0.9, n_estimators=100, device="cuda", tree_method="hist"),
        n_jobs=-1
    )
    print("Fitting stack_model - Start", datetime.now())
    stack_model.fit(X_train_fold, y_train_fold)
    print("Fitting stack_model - End", datetime.now())

    train_score = root_mean_squared_log_error(np.expm1(stack_model.predict(X_train_fold)), np.expm1(y_train_fold))
    val_score = root_mean_squared_log_error(np.expm1(stack_model.predict(X_val_fold)), np.expm1(y_val_fold))

    print(f"Train: {train_score}, Test: {val_score}")

Starting
Fitting catboost - Start 2024-12-29 02:10:21.378310
Fitting catboost - End 2024-12-29 02:10:52.899307
Fitting xgboost - Start 2024-12-29 02:10:53.141309
Fitting xgboost - End 2024-12-29 02:10:56.184306
Fitting xgboost_rf - Start 2024-12-29 02:10:56.184306
Fitting xgboost_rf - End 2024-12-29 02:10:59.613306
Fitting lgbm - Start 2024-12-29 02:10:59.613306
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037577 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4800
[LightGBM] [Info] Number of data points in the train set: 960000

d:\Development\insurance\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [02:11:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Fitting stack_model - End 2024-12-29 02:11:05.478308
Train: 1.0220528175630508, Test: 1.0442864042947442
Starting
Fitting catboost - Start 2024-12-29 02:11:08.779307


KeyboardInterrupt: 

In [ ]:
full_pipeline.fit(treated_dataset, treated_dataset['Premium Amount'])


In [16]:
res = full_pipeline.predict(treated_dataset)


In [17]:
from utils.metrics import rmsle_metric, source_range_root_mean_squared_log_error

In [ ]:
source_range_root_mean_squared_log_error(treated_dataset['Premium Amount'], res)


In [ ]:
pd.Series(res).hist()

In [7]:
import miceforest as mf

In [ ]:
kernel = mf.ImputationKernel(treated_dataset, num_datasets=1, random_state=1)


In [46]:
from optbinning import ContinuousOptimalBinning


In [ ]:
ContinuousOptimalBinning?

In [ ]:
binning = ContinuousOptimalBinning(name="Premium Amount", dtype="numerical")
binning.fit(treated_dataset["Credit Score"], treated_dataset["Premium Amount"])



In [ ]:
binning.binning_table.plot()


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import ShuffleSplit
from datetime import datetime
import numpy as np

# Define objective function
def objective(trial):
    param_grid = {
        'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
        'objective': 'regression',  # Use 'binary' for classification
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.02, 0.1),  # Fine-tuned range
        'num_leaves': trial.suggest_int('num_leaves', 1, 50),  # Wider range for more exploration
        'max_depth': trial.suggest_int('max_depth', -1, 50),  # Allow deeper trees or unlimited depth (-1)
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 0.8),  # Sampling fraction of features
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.85, 1.0),  # Sampling fraction of data
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),  # Frequency of bagging
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),  # Minimum samples per leaf
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-3, 0.1),  # Minimum gain for a split
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-3, 10),  # Minimum weight for children
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10),  # L1 regularization
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10),  # L2 regularization
        'cat_smooth': trial.suggest_uniform('cat_smooth', 5, 30),  # Smoothing for categorical features
        'max_bin': trial.suggest_int('max_bin', 150, 200),  # Granularity of feature splits
        'verbose': -1,  # Suppress output
        'random_state': 41,  # Reproducibility
    }
    print(param_grid)
    
    # Define a 60-40 split
    splitter = ShuffleSplit(n_splits=5, test_size=0.4, random_state=1)


    results = []
    for train_idx, val_idx in splitter.split(X_train_fixed):
        # Prepare data for LightGBM
        X_train_fold, X_val_fold = X_train_fixed.iloc[train_idx], X_train_fixed.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        train_data_lgbm = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data_lgbm = lgb.Dataset(X_val_fold, label=y_val_fold)
        
        # Train model
        model = lgb.train(
            param_grid,
            train_data_lgbm,
            num_boost_round=500,
            valid_sets=[train_data_lgbm, val_data_lgbm],
            valid_names=["train", "validation"],
            feval=rmsle_metric,    # Custom RMSLE metric
            callbacks=[
                lgb.early_stopping(stopping_rounds=10),
                lgb.log_evaluation(period=5),  # Optional: Logs evaluation every 10 rounds
            ]
        )
        results.append(model.best_score['validation']['rmsle'])
        
    # Use validation score
    return np.mean(results)